<a href="https://colab.research.google.com/github/darshil89/RAG/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install -U --upgrade langchain langchain-openai langchain-core langchain_community docx2txt pypdf langchain_chroma sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.3/423.3 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 69.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 340.6/340.6 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 29.1 MB/s eta 0

In [5]:
import os
os.environ["OPENAI_API_KEY"]="XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]="langchain-basics"
os.environ["LANGCHAIN_API_KEY"]="XXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"

In [6]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")
llm_response = llm.invoke("Tell me a joke?")
llm_response

AIMessage(content='Why did the scarecrow win an award? \n\nBecause he was outstanding in his field!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 12, 'total_tokens': 31, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_44added55e', 'id': 'chatcmpl-BKcK6IKFQtp8eSoykRc4qmyRph8oU', 'finish_reason': 'stop', 'logprobs': None}, id='run-fe97fda0-f485-4b7d-bb69-2835c756fea3-0', usage_metadata={'input_tokens': 12, 'output_tokens': 19, 'total_tokens': 31, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [7]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()
output_parser.invoke(llm_response)

'Why did the scarecrow win an award? \n\nBecause he was outstanding in his field!'

In [8]:
chain = llm | output_parser
chain.invoke("What should i call you?")

"You can call me Assistant! If you have a specific name you'd like to use, feel free to choose one."

In [9]:
from typing import List
from pydantic import BaseModel, Field

class MobileReview(BaseModel):
  phone_model: str = Field(description="Name and model of the phone")
  rating: float= Field(description="Overall rating out of 5")
  pros: List[str] = Field(description="List of pros")
  cons: List[str] = Field(description="List of cons")
  summary : str = Field(description="Review summary")


review_text = """
Just got my hands on the new Galaxy S21 and wow, this thing is slick! The screen is gorgeous, colors pop like crazy. Camera's insane too, especially at night - my Insta game's never been stronger. Battery life's solid, lasts me all day no problem.

Not gonna lie though, it's pretty pricey. And what's with ditching the charger? C'mon Samsung. Also, still getting used to the new button layout, keep hitting Bixby by mistake.

Overall, I'd say it's a solid 4 out of 5. Great phone, but a few annoying quirks keep it from being perfect. If you're due for an upgrade, definitely worth checking out!
"""

structured_llm = llm.with_structured_output(MobileReview)
output = structured_llm.invoke(review_text)
output

MobileReview(phone_model='Samsung Galaxy S21', rating=4.0, pros=['Gorgeous screen with vibrant colors', 'Insane camera performance, especially at night', 'Solid battery life lasting all day'], cons=['Expensive price point', 'No charger included', 'New button layout can be confusing'], summary='The Samsung Galaxy S21 is a fantastic phone with a stunning display and impressive camera performance, making it great for photography enthusiasts. However, its high price and lack of included charger, coupled with some design quirks, prevent it from reaching perfection.')

In [10]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("Tell me something about {topic}")
prompt.invoke({"topic":"dogs"})

ChatPromptValue(messages=[HumanMessage(content='Tell me something about dogs', additional_kwargs={}, response_metadata={})])

In [11]:
chain = prompt | llm | output_parser
chain.invoke({"topic":"dogs"})

'Sure! Dogs are known as "man\'s best friend" for many reasons. They are domesticated mammals, descended from wolves, and have been companions to humans for thousands of years. Here are some interesting facts about dogs:\n\n1. **Diversity**: There are hundreds of dog breeds, each with unique characteristics, sizes, and temperaments. For example, the Chihuahua is one of the smallest breeds, while the Great Dane can be one of the tallest.\n\n2. **Communication**: Dogs communicate through a combination of vocalizations (barking, howling, growling), body language (tail position, ear posture), and facial expressions. They are also adept at reading human emotions and cues.\n\n3. **Sense of Smell**: Dogs have an extraordinary sense of smell, which is estimated to be between 10,000 to 100,000 times more sensitive than that of humans. This makes them excellent in roles such as search and rescue, detecting drugs, and even identifying certain medical conditions.\n\n4. **Intelligence and Training*

# LLM Messages

In [12]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, SystemMessage

system_message = SystemMessage(content="You are a nice AI bot that helps a user figure out what to eat in one short sentence")
human_message = HumanMessage(content="I like tomatoes, what should I eat?")
llm.invoke([system_message, human_message])

AIMessage(content='How about a fresh caprese salad with tomatoes, mozzarella, basil, and a drizzle of balsamic glaze?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 39, 'total_tokens': 62, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_64e0ac9789', 'id': 'chatcmpl-BKcKHeNkK5RfFIV5jK1lHGhXGmgSr', 'finish_reason': 'stop', 'logprobs': None}, id='run-453ea00b-bba4-4ab4-8049-7cda45edabab-0', usage_metadata={'input_tokens': 39, 'output_tokens': 23, 'total_tokens': 62, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

# RAG

In [13]:
!pip install -U langchain-community langchain-openai docx2txt

In [14]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Split the data into chunks

In [15]:
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from typing import List
from langchain_core.documents import Document

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=200,
    length_function=len,
)

docx_loader = Docx2txtLoader("/content/drive/MyDrive/dummy_data/Employee_handbook.docx")
documents = docx_loader.load()

splits = text_splitter.split_documents(documents)

print(f"Number of splits: {len(splits)} chunks")

Number of splits: 2 chunks


For multiple files

In [16]:
def load_documents(folder_path: str) -> List[documents]:
  documents = []
  for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    if filename.endswith(".pdf"):
      loader = PyPDFLoader(file_path)
    elif filename.endswith(".docx") or filename.endswith(".doc"):
      loader = Docx2txtLoader(file_path)
    else:
      continue
    documents.extend(loader.load())

  return documents


folder_path = "/content/drive/MyDrive/dummy_data"
documents = load_documents(folder_path)

print(f"Loaded {len(documents)} documents from folder")
splits = text_splitter.split_documents(documents)
print(f"Split the documents into {len(splits)} chunks")

Loaded 4 documents from folder
Split the documents into 7 chunks


create embeddings

In [17]:
embeddings = OpenAIEmbeddings()

documents_embeddings = embeddings.embed_documents([doc.page_content for doc in splits])

print(f"Created embeddings for {len(documents_embeddings)} documents")

Created embeddings for 7 documents


Store the embedding in the vector store

In [18]:
from langchain_chroma import Chroma

persist_directory = "./chroma_db"
embedding_function = OpenAIEmbeddings()

vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=embedding_function,
    persist_directory=persist_directory
)

print("✅ Chroma vectorstore created and saved at", persist_directory)


✅ Chroma vectorstore created and saved at ./chroma_db


By now the store is completed and now lets test on data

In [19]:
from langchain_core.documents import Document

query = "When does the office open?"
results = vectorstore.similarity_search(query, k=2)

for i, doc in enumerate(results, 1):
    print(f"\n📄 Match {i}:")
    print(doc.page_content)



📄 Match 1:
Company Name: Novex Technologies Pvt. Ltd.



Welcome to Novex Technologies! We're excited to have you on board. This handbook provides a comprehensive overview of our company policies, work culture, and employee expectations.



1. Office Hours: 9:00 AM - 6:00 PM, Monday to Friday

2. Remote Work Policy: Employees may work remotely up to 2 days per week upon manager approval.

3. Leave Policy:

   - Annual Leave: 20 days per year

   - Sick Leave: 10 days per year

📄 Match 2:
2. Remote Work Policy: Employees may work remotely up to 2 days per week upon manager approval.

3. Leave Policy:

   - Annual Leave: 20 days per year

   - Sick Leave: 10 days per year

   - Casual Leave: 5 days per year

4. Code of Conduct:

   - Respect colleagues and customers.

   - Maintain confidentiality of company data.

   - Avoid conflicts of interest.



For questions, contact hr@novextech.com


By using retriver object

In [20]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

retriever.invoke("What are the office timings?")


[Document(id='75f52866-0b0c-4e6b-bd0c-2b1f50ac0184', metadata={'source': '/content/drive/MyDrive/dummy_data/Employee_handbook.docx'}, page_content="Company Name: Novex Technologies Pvt. Ltd.\n\n\n\nWelcome to Novex Technologies! We're excited to have you on board. This handbook provides a comprehensive overview of our company policies, work culture, and employee expectations.\n\n\n\n1. Office Hours: 9:00 AM - 6:00 PM, Monday to Friday\n\n2. Remote Work Policy: Employees may work remotely up to 2 days per week upon manager approval.\n\n3. Leave Policy:\n\n   - Annual Leave: 20 days per year\n\n   - Sick Leave: 10 days per year"),
 Document(id='6b9bbae2-b824-4051-9293-d33841755a32', metadata={'source': '/content/drive/MyDrive/dummy_data/Employee_handbook.docx'}, page_content='2. Remote Work Policy: Employees may work remotely up to 2 days per week upon manager approval.\n\n3. Leave Policy:\n\n   - Annual Leave: 20 days per year\n\n   - Sick Leave: 10 days per year\n\n   - Casual Leave: 5

Create a prompt

In [21]:
from langchain_core.prompts import ChatPromptTemplate
template = """ Answer the question based only on the following context:
{context}

Question: {question}

Answer: """

prompt = ChatPromptTemplate.from_template(template)

In [22]:
def doc2str(docs):
  return "\n\n".join(doc.page_content for doc in docs)

Now using our doc2str function

In [23]:
from langchain.schema.runnable import RunnablePassthrough


rag_chain = (
    {"context": retriever | doc2str , "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
question = "What are the regular working hours at Novex Technologies?"
response = rag_chain.invoke(question)
response

'The regular working hours at Novex Technologies are from 9:00 AM to 6:00 PM, Monday to Friday.'

**Create a chat history for follow up questions -- to make a conversational RAG**

In [24]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = []
chat_history.extend([
    HumanMessage(content=question),
    AIMessage(content=response)
])
chat_history

[HumanMessage(content='What are the regular working hours at Novex Technologies?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='The regular working hours at Novex Technologies are from 9:00 AM to 6:00 PM, Monday to Friday.', additional_kwargs={}, response_metadata={})]

This is to contectlize the question.

In [25]:
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt = (
    "Given a chat history and the latest user question"
    "which might reference context in the chat history."
    "formulate a stand alone question which can be understood"
    "without the chat history dont answer the question"
    "just reformulate it if needed and otherwise return it as is"
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

contextualize_q_chain = (
    contextualize_q_prompt
    | llm
    | StrOutputParser()
)

contextualize_q_chain.invoke({"chat_history": chat_history, "input": "Is the office open on weekends?"})



'Is the office at Novex Technologies open on weekends?'

Now use this prompt to call the llm

In [26]:
from langchain.chains import create_history_aware_retriever

history_aware_retriever = create_history_aware_retriever(
    llm,
    retriever,
    contextualize_q_prompt,
)

history_aware_retriever.invoke({"input":"Is the office open on weekends?", "chat_history": chat_history})

[Document(id='75f52866-0b0c-4e6b-bd0c-2b1f50ac0184', metadata={'source': '/content/drive/MyDrive/dummy_data/Employee_handbook.docx'}, page_content="Company Name: Novex Technologies Pvt. Ltd.\n\n\n\nWelcome to Novex Technologies! We're excited to have you on board. This handbook provides a comprehensive overview of our company policies, work culture, and employee expectations.\n\n\n\n1. Office Hours: 9:00 AM - 6:00 PM, Monday to Friday\n\n2. Remote Work Policy: Employees may work remotely up to 2 days per week upon manager approval.\n\n3. Leave Policy:\n\n   - Annual Leave: 20 days per year\n\n   - Sick Leave: 10 days per year"),
 Document(id='6b9bbae2-b824-4051-9293-d33841755a32', metadata={'source': '/content/drive/MyDrive/dummy_data/Employee_handbook.docx'}, page_content='2. Remote Work Policy: Employees may work remotely up to 2 days per week upon manager approval.\n\n3. Leave Policy:\n\n   - Annual Leave: 20 days per year\n\n   - Sick Leave: 10 days per year\n\n   - Casual Leave: 5

In [27]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import MessagesPlaceholder

qa_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an helpful AI assistant. Use the following context to answer the user's question."),
    ("system", "Context: {context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
])

question_answer_chain = create_stuff_documents_chain(
    llm,
    qa_prompt,)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

rag_chain.invoke({"input":"Is the office open on weekends?", "chat_history": chat_history})

{'input': 'Is the office open on weekends?',
 'chat_history': [HumanMessage(content='What are the regular working hours at Novex Technologies?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The regular working hours at Novex Technologies are from 9:00 AM to 6:00 PM, Monday to Friday.', additional_kwargs={}, response_metadata={})],
 'context': [Document(id='75f52866-0b0c-4e6b-bd0c-2b1f50ac0184', metadata={'source': '/content/drive/MyDrive/dummy_data/Employee_handbook.docx'}, page_content="Company Name: Novex Technologies Pvt. Ltd.\n\n\n\nWelcome to Novex Technologies! We're excited to have you on board. This handbook provides a comprehensive overview of our company policies, work culture, and employee expectations.\n\n\n\n1. Office Hours: 9:00 AM - 6:00 PM, Monday to Friday\n\n2. Remote Work Policy: Employees may work remotely up to 2 days per week upon manager approval.\n\n3. Leave Policy:\n\n   - Annual Leave: 20 days per year\n\n   - Sick Leave: 10 days per year"

# Lets go to Production

In [28]:
import sqlite3
from datetime import datetime
import uuid

DB_NAME = "rag_app.db"

def get_db_connection():
    conn = sqlite3.connect(DB_NAME)
    conn.row_factory = sqlite3.Row
    return conn

def create_application_logs():
    conn = get_db_connection()
    conn.execute('''CREATE TABLE IF NOT EXISTS application_logs
    (id INTEGER PRIMARY KEY AUTOINCREMENT,
    session_id TEXT,
    user_query TEXT,
    gpt_response TEXT,
    model TEXT,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP)''')
    conn.close()

def insert_application_logs(session_id, user_query, gpt_response, model):
    conn = get_db_connection()
    conn.execute('INSERT INTO application_logs (session_id, user_query, gpt_response, model) VALUES (?, ?, ?, ?)',
                 (session_id, user_query, gpt_response, model))
    conn.commit()
    conn.close()

def get_chat_history(session_id):
    conn = get_db_connection()
    cursor = conn.cursor()
    cursor.execute('SELECT user_query, gpt_response FROM application_logs WHERE session_id = ? ORDER BY created_at', (session_id,))
    messages = []
    for row in cursor.fetchall():
        messages.extend([
            {"role": "human", "content": row['user_query']},
            {"role": "ai", "content": row['gpt_response']}
        ])
    conn.close()
    return messages

# Initialize the database
create_application_logs()

In [30]:
# Example usage for a new user
session_id = str(uuid.uuid4())
question = "What is Novex Technologies?"
chat_history = get_chat_history(session_id)
answer = rag_chain.invoke({"input": question, "chat_history": chat_history})['answer']
insert_application_logs(session_id, question, answer, "gpt-3.5-turbo")
print(f"Human: {question}")
print(f"AI: {answer}\n")

# Example of a follow-up question
question2 = "What was their first product?"
chat_history = get_chat_history(session_id)
answer2 = rag_chain.invoke({"input": question2, "chat_history": chat_history})['answer']
insert_application_logs(session_id, question2, answer2, "gpt-3.5-turbo")
print(f"Human: {question2}")
print(f"AI: {answer2}")


Human: What is Novex Technologies?
AI: Novex Technologies Pvt. Ltd. is a company that specializes in providing enterprise-ready solutions, particularly through its platform, the Novex AI Suite. This suite helps businesses automate workflows, extract insights from data, and build intelligent systems, integrating various key modules such as AI Assistant Pro, DocuParse, and InsightViz. The company focuses on reducing manual workload, improving customer response times, and enhancing business efficiency through advanced technology.

Human: What was their first product?
AI: The context provided does not specify which product was the first for Novex Technologies Pvt. Ltd. However, the key product mentioned is the Novex AI Suite, which includes modules like AI Assistant Pro, DocuParse, and InsightViz. If you're looking for information about specific product launches or their order, that detail is not included in the available context.
